<a href="https://colab.research.google.com/github/Ckiteme/CKiteme-Assignment-Apache-Spark-DataFrames-Project/blob/main/CKiteme_Assignment_Apache_Spark_DataFrames_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='#2F4F4F'>1. Problem Statement</font>

**Instructions**

As a Data professional, you need to perform an analysis by answering questions about
some stock market data on Safaricom from the years 2012-2017.
You will need to perform the following:

**Data Importation and Exploration**

● Start a spark session and load the stock file while inferring the data types.
● Determine the column names
● Make observations about the schema.
● Show the first 5 rows
● Use the describe method to learn about the data frame

**Data Preparation**

● Format all the data to 2 decimal places i.e. format_number()
● Create a new data frame with a column called HV Ratio that is the ratio of the
High Price versus volume of stock traded for a day
Data Analysis
● What day had the Peak High in Price?
● What is the mean of the Close column?
● What is the max and min of the Volume column?
● How many days was the Close lower than 60 dollars?
● What percentage of the time was the High greater than 80 dollars?
● What is the Pearson correlation between High and Volume?
● What is the max High per year?
● What is the average Close for each Calendar Month?

# <font color='#2F4F4F'>2. Data Importation and Exploration</font>

In [2]:
# Installing pyspark
# ---
#
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 43 kB/s 
     |████████████████████████████████| 199 kB 60.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=03086bde4bcdc29eaa7b403a3d2629e4b9cbdb4b86a4a70cba92f85d15c5d244
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [3]:
# Next, we run a local spark session
# ---
#
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
sc = spark.sparkContext

import warnings
warnings.filterwarnings('ignore')

from pyspark.sql import SQLContext
sqlCtx = SQLContext(sc)

In [4]:
#read csv and display schema to determine the column names
df = spark.read.csv("saf_stock.csv", header=True)
print(df.columns)
df.printSchema()

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']
root
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



**Observations:**

There are 7 columns

Columns are nullable

Column type is String

The first row is a header that contains the column names

In [5]:
#Show the first 5 rows

df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



In [6]:
#Use the describe method to learn about the data frame
df.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|         10010500|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700


**3. Data Preparation**

In [7]:
#Format all the data to 2 decimal places

from pyspark.sql.functions import format_number

df_formatted=df.select(df['Date'],
              format_number(df['Open'].cast('float'),2).alias('Open'),
              format_number(df['High'].cast('float'), 2).alias('High'),
              format_number(df['Low'].cast('float'), 2).alias('Low'),
              format_number(df['Close'].cast('float'), 2).alias('Close'),
              df['Volume'].cast('int').alias('Volume'),
              format_number(df['Adj Close'].cast('float'), 2).alias('Adj Close')
              )


from pyspark.sql import functions as F

df_formatted.show()

+----------+-----+-----+-----+-----+--------+---------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|
+----------+-----+-----+-----+-----+--------+---------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|    52.62|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|    52.08|
|2012-01-05|59.35|59.62|58.37|59.42|12768200|    51.83|
|2012-01-06|59.42|59.45|58.87|59.00| 8069400|    51.46|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|    51.62|
|2012-01-10|59.43|59.71|58.98|59.04| 6907300|    51.49|
|2012-01-11|59.06|59.53|59.04|59.40| 6365600|    51.81|
|2012-01-12|59.79|60.00|59.40|59.50| 7236400|    51.90|
|2012-01-13|59.18|59.61|59.01|59.54| 7729300|    51.93|
|2012-01-17|59.87|60.11|59.52|59.85| 8500000|    52.20|
|2012-01-18|59.79|60.03|59.65|60.01| 5911400|    52.34|
|2012-01-19|59.93|60.73|59.75|60.61| 9234600|    52.86|
|2012-01-20|60.75|61.25|60.67|61.01|10378800|    53.21|
|2012-01-23|60.81|60.98|60.51|60.91| 7134100|    53.13|
|2012-01-24|60.75|62.00|60.75|61.39| 7362800|   

In [8]:
# create a new data frame with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day
df_new = df_formatted.withColumn('HV Ratio', df_formatted['High']/ df_formatted['Volume'])
df_new.show()

+----------+-----+-----+-----+-----+--------+---------+--------------------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|            HV Ratio|
+----------+-----+-----+-----+-----+--------+---------+--------------------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|    52.62|4.819714574387472E-6|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|    52.08|6.290848821573389...|
|2012-01-05|59.35|59.62|58.37|59.42|12768200|    51.83|4.669413073103491E-6|
|2012-01-06|59.42|59.45|58.87|59.00| 8069400|    51.46|7.367338339901356E-6|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|    51.62|8.915604928660188E-6|
|2012-01-10|59.43|59.71|58.98|59.04| 6907300|    51.49|8.644477581688938E-6|
|2012-01-11|59.06|59.53|59.04|59.40| 6365600|    51.81| 9.35182857861003E-6|
|2012-01-12|59.79|60.00|59.40|59.50| 7236400|    51.90| 8.29141562102703E-6|
|2012-01-13|59.18|59.61|59.01|59.54| 7729300|    51.93|7.712211972623653E-6|
|2012-01-17|59.87|60.11|59.52|59.85| 8500000|    52.20|7.071764705882352...|

**4. Data Analysis**

In [9]:
#What day had the Peak High in Price?
print(df_new.orderBy(df_new['High'].desc()).head(1)[0][0])

2015-01-13


In [10]:
#What is the mean of the Close column?
from pyspark.sql.functions import mean, max, min

print(df_new.select(mean('Close')).show())

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844992050863|
+-----------------+

None


In [11]:
#What is the max and min of the Volume column?
print(df_new.select(max('Volume') ,min('Volume')).show())

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+

None


In [12]:
#How many days was the Close lower than 60 dollars?
print(df_new.filter(df_new['Close'] < 60).count())

81


In [13]:
#What percentage of the time was the High greater than 80 dollars

print((df_new.filter(df_new['High']>80).count()/df.count()) * 100)

8.426073131955485


In [14]:
#What is the Pearson correlation between High and Volume?
from pyspark.sql.functions import corr
print(df_new.select(corr('High','Volume')).show())

+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.33843260582148915|
+--------------------+

None


In [15]:
# What is the max High per year?

from pyspark.sql.functions import year

df_new_2=df_new.select(df_new['Date'],
              df_new['High'].cast('float').alias('High'),
              df_new['Close'].cast('float').alias('Close')
              )

df_year = df_new_2.withColumn("Year",year(df_new_2['Date']))
maxim= df_year.groupBy('Year').max()
print(maxim.select('Year','max(High)').orderBy('Year').show())

+----+---------+
|Year|max(High)|
+----+---------+
|2012|     77.6|
|2013|    81.37|
|2014|    88.09|
|2015|    90.97|
|2016|    75.19|
+----+---------+

None


In [16]:
from pyspark.sql.functions import month

df_month = df_new_2.withColumn('Month',month('Date'))
avg_month = df_month.select(['Month','Close']).groupBy('Month').mean()
print(avg_month.select('Month','avg(Close)').orderBy('Month').show())

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1| 71.4480196131338|
|    2|71.30680438169499|
|    3|71.77794376266337|
|    4|72.97361900692894|
|    5|72.30971685445533|
|    6| 72.4953774506191|
|    7|74.43971944078106|
|    8| 73.0298185521906|
|    9|72.18411782208611|
|   10| 71.5785454489968|
|   11|72.11108927207418|
|   12|72.84792482628012|
+-----+-----------------+

None
